![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FDev%2FTools&file=dataproc-serverless-pyspark.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Dev/Tools/dataproc-serverless-pyspark.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FDev%2FTools%2Fdataproc-serverless-pyspark.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Dev/Tools/dataproc-serverless-pyspark.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Dev/Tools/dataproc-serverless-pyspark.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

## Spark Jobs with Dataproc Serverless

[Dataproc Serverless](https://cloud.google.com/dataproc-serverless/docs/overview) runs Spark jobs as batch workloads in a managed infrastructure that [autoscales](https://cloud.google.com/dataproc-serverless/docs/concepts/autoscaling) resources as needed.  Simply put, all you need is a job!  

This allows you to run PySpark, Spark SQL, Spark R, Spark Jave/Scala.  You can set most Spark Properties, including these [resource allocation properties](https://cloud.google.com/dataproc-serverless/docs/concepts/properties) to determine compute, memory, and disk resources for initial active executors and autoscaling maximums.  You can also use [custom containers](https://cloud.google.com/dataproc-serverless/docs/guides/custom-containers) without the need to include Spark which will be mounted to the container at runtime.

**Overview**

The example below shows the process of setting up a GCP environment and submitting a Dataproc Serverless PySpark job.  This job uses the provided BigQuery connector to read from BigQuery, process the data with Spark, then write the result to a BigQuery table.


## Environment Setup

### Create Parameters

In [1]:
# Defined Parameters
PROJECT_ID = 'statmike-demo3'
NOTEBOOK = 'dataproc'
REGION = 'us-central1'

# Derived Parameters
GCS_BUCKET = PROJECT_ID
BQ_DATASET = NOTEBOOK
GCS_FOLDER = f'demos/{NOTEBOOK}'

### Import Libraries

In [10]:
from google.cloud import bigquery
from google.cloud import storage

### Setup Clients

In [11]:
bq = bigquery.Client(project = PROJECT_ID)
gcs = storage.Client()

### Local Directory

In [12]:
DIR = NOTEBOOK
!rm -rf {NOTEBOOK}
!mkdir -p {NOTEBOOK}

### GCS Bucket

In [13]:
buckets = !gsutil list -p {PROJECT_ID}
if f"gs://{GCS_BUCKET}/" not in buckets:
    ! gsutil mb -l us -c standard gs://{GCS_BUCKET}
else: print(f"Bucket gs://{GCS_BUCKET} already exists")

Bucket gs://statmike-demo3 already exists


### BigQuery Dataset

In [15]:
ds = bigquery.Dataset(f"{PROJECT_ID}.{BQ_DATASET}")
ds.location = 'US'
ds = bq.create_dataset(dataset = ds, exists_ok = True)

### Setup Dataproc
Using Google APIs from Spark code will require the subnet to have Private Google Access enabled.
- Network Configuration: https://cloud.google.com/dataproc-serverless/docs/concepts/network
    - Configure Private Google Access: https://cloud.google.com/vpc/docs/configure-private-google-access#config-pga

In [17]:
status = !gcloud compute networks subnets describe default --region={REGION} --format="get(privateIpGoogleAccess)"
if status[0] == 'False':
  !gcloud compute networks subnets update default --region={REGION} --enable-private-ip-google-access
  status = !gcloud compute networks subnets describe default --region={REGION} --format="get(privateIpGoogleAccess)"
print(f"Private Google Access is Enable = {status[0]}")

Updated [https://www.googleapis.com/compute/v1/projects/statmike-demo3/regions/us-central1/subnetworks/default].
Private Google Access is Enable = True


## Dataproc Serverless Spark Batch Job
- Dataproc Serverless: https://cloud.google.com/dataproc-serverless/docs/overview
- BigQuery Connector: https://github.com/GoogleCloudDataproc/spark-bigquery-connector
- gcloud dataproc batches submit pyspark [documentation](https://cloud.google.com/sdk/gcloud/reference/dataproc/batches/submit/pyspark)

### Define PySpark Job

In [18]:
%%writefile {DIR}/myjob.py
#!/usr/bin/python
"""BigQuery I/O PySpark example."""
from pyspark.sql import SparkSession
import sys

print("Number of Arguments: {0} arguments.".format(len(sys.argv)))
print("Arguments List: {0}".format(str(sys.argv)))

# create a session
spark = SparkSession.builder.appName('spark-bigquery').getOrCreate()

# Use the Cloud Storage bucket for temporary BigQuery export data used by the connector.
spark.conf.set('temporaryGcsBucket', sys.argv[1])

# Load data from BigQuery.
words = spark.read.format('bigquery').option('table', 'bigquery-public-data:samples.shakespeare').load()
# Create a View
words.createOrReplaceTempView('words')

# Perform word count.
word_count = spark.sql('SELECT word, SUM(word_count) AS word_count FROM words GROUP BY word ORDER BY word_count DESC')
word_count.show(n=5)
word_count.printSchema()

# Saving the data to BigQuery
word_count.write.format('bigquery').option('table', sys.argv[2]).mode('overwrite').save()

Writing dataproc/myjob.py


### Run PySpark Job

In [19]:
bq.query(query = f"SELECT COUNT(*) as record_count FROM bigquery-public-data.samples.shakespeare").to_dataframe()['record_count'].iloc[0]

164656

In [20]:
!gcloud dataproc batches submit pyspark {DIR}/myjob.py \
--project={PROJECT_ID} \
--region={REGION} \
--deps-bucket={GCS_BUCKET} \
--jars=gs://spark-lib/bigquery/spark-bigquery-with-dependencies_2.12-0.24.2.jar \
-- {GCS_BUCKET}/{GCS_FOLDER} \
    {PROJECT_ID}:{BQ_DATASET}.myjob_output

Batch [55de12091ced4183b5f62195999fa0bd] submitted.
Using the default container image
PYSPARK_PYTHON=/opt/dataproc/conda/bin/python
JAVA_HOME=/usr/lib/jvm/temurin-11-jdk-amd64
SPARK_EXTRA_CLASSPATH=
:: loading settings :: file = /etc/spark/conf/ivysettings.xml
Number of Arguments: 3 arguments.
Arguments List: ['/tmp/srvls-batch-1e378212-be7e-4f71-8130-ea9eac086eef/myjob.py', 'statmike-demo3/demos/dataproc', 'statmike-demo3:dataproc.myjob_output']
22/05/28 13:04:36 INFO DirectBigQueryRelation: Querying table bigquery-public-data.samples.shakespeare, parameters sent from Spark: requiredColumns=[word,word_count], filters=[]
22/05/28 13:04:36 INFO DirectBigQueryRelation: Going to read from bigquery-public-data.samples.shakespeare columns=[word, word_count], filter=''
22/05/28 13:04:38 INFO DirectBigQueryRelation: Created read session for table 'bigquery-public-data.samples.shakespeare': projects/statmike-demo3/locations/us/sessions/CAISDEUwNEpyaDR1QXFNTxoCamQaAmpmGgJpchoCb2oaAmpxGgJuYRoCb3

In [21]:
bq.query(query = f"SELECT * FROM {PROJECT_ID}.{NOTEBOOK}.myjob_output ORDER BY word_count DESC LIMIT 5").to_dataframe()

,word,word_count
0,the,25568
1,I,21028
2,and,19649
3,to,17361
4,of,16438
